In [2]:
from glob import glob
import xarray as xr
import logging
import os
from datetime import datetime

from geoips.interfaces import readers, algorithms, interpolators, sectors, colormappers, output_formatters, gridline_annotators, feature_annotators

LOG = logging.getLogger(__name__)

abi_reader = readers.get_plugin("abi_netcdf")

single_channel_algorithm = algorithms.get_plugin("single_channel")

interp_nearest = interpolators.get_plugin("interp_nearest")

conus_sector_area = sectors.get_plugin("conus").area_definition

in_fpath = f"{os.environ['GEOIPS_TESTDATA_DIR']}/test_data_noaa_aws/data/goes16/20210929/0000/*.nc"
in_fpaths = glob(in_fpath)

In [3]:
print("Reading abi data.")
xdict = abi_reader(fnames=in_fpaths, area_def=conus_sector_area, chans=["B14BT"])

conus_xobj = xdict["conus"]

conus_xobj

Reading abi data.


<xarray.Dataset> Size: 161MB
Dimensions:                  (dim_0: 1200, dim_1: 2400)
Dimensions without coordinates: dim_0, dim_1
Data variables:
    B14BT                    (dim_0, dim_1) float64 23MB 260.2 260.7 ... 294.1
    latitude                 (dim_0, dim_1) float64 23MB nan nan ... 23.33 23.33
    longitude                (dim_0, dim_1) float64 23MB nan nan ... -65.99
    satellite_zenith_angle   (dim_0, dim_1) float64 23MB nan nan ... 29.26 29.27
    satellite_azimuth_angle  (dim_0, dim_1) float64 23MB nan nan ... 21.78 21.83
    solar_zenith_angle       (dim_0, dim_1) float64 23MB nan nan ... 113.3 113.3
    solar_azimuth_angle      (dim_0, dim_1) float64 23MB nan nan ... 4.853 4.854
Attributes:
    file_metadata:                      {'B14': {'file_info': {'id': '0487a2b...
    start_datetime:                     2021-09-29 00:00:20.634654
    end_datetime:                       2021-09-29 00:09:51.432276
    source_name:                        abi
    data_provider:                      noaa
    source_file_names:                  ['OR_ABI-L1b-RadF-M6C14_G16_s20212720...
    platform_name:                      goes-16
    area_definition:                    Area ID: conus\nDescription: Continen...
    area_id:                            conus
    interpolation_radius_of_influence:  10000.0

In [4]:
print("Applying interp_nearest interpolator.")
output_dataset = interp_nearest(conus_sector_area, conus_xobj, None, ["B14BT", "longitude", "latitude"])

output_dataset

Applying interp_nearest interpolator.


<xarray.Dataset> Size: 69MB
Dimensions:    (dim_0: 1200, dim_1: 2400)
Dimensions without coordinates: dim_0, dim_1
Data variables:
    latitude   (dim_0, dim_1) float64 23MB nan nan nan nan ... 23.33 23.33 23.33
    longitude  (dim_0, dim_1) float64 23MB nan nan nan ... -66.04 -66.02 -65.99
    B14BT      (dim_0, dim_1) float64 23MB nan nan nan nan ... 294.0 293.8 294.1
Attributes:
    start_datetime:                     2021-09-29 00:00:20.634654
    end_datetime:                       2021-09-29 00:09:51.432276
    platform_name:                      goes-16
    source_name:                        abi
    data_provider:                      noaa
    source_file_names:                  ['OR_ABI-L1b-RadF-M6C14_G16_s20212720...
    interpolation_radius_of_influence:  10000.0
    area_definition:                    Area ID: conus\nDescription: Continen...
    registered_dataset:                 True

In [5]:
print("Applying single_channel algorithm.")
channel_14_bt = output_dataset["B14BT"].data

algorithm_output = single_channel_algorithm([channel_14_bt], output_data_range=[-90.0, 30.0], input_units="Kelvin", output_units="celsius")

output_dataset["Infrared"] = xr.DataArray(algorithm_output)

output_dataset

Applying single_channel algorithm.


<xarray.Dataset> Size: 92MB
Dimensions:    (dim_0: 1200, dim_1: 2400)
Dimensions without coordinates: dim_0, dim_1
Data variables:
    latitude   (dim_0, dim_1) float64 23MB nan nan nan nan ... 23.33 23.33 23.33
    longitude  (dim_0, dim_1) float64 23MB nan nan nan ... -66.04 -66.02 -65.99
    B14BT      (dim_0, dim_1) float64 23MB nan nan nan nan ... 294.0 293.8 294.1
    Infrared   (dim_0, dim_1) float64 23MB nan nan nan nan ... 20.81 20.66 20.99
Attributes:
    start_datetime:                     2021-09-29 00:00:20.634654
    end_datetime:                       2021-09-29 00:09:51.432276
    platform_name:                      goes-16
    source_name:                        abi
    data_provider:                      noaa
    source_file_names:                  ['OR_ABI-L1b-RadF-M6C14_G16_s20212720...
    interpolation_radius_of_influence:  10000.0
    area_definition:                    Area ID: conus\nDescription: Continen...
    registered_dataset:                 True

In [6]:
ir_colormapper = colormappers.get_plugin("Infrared")

ir_color_dict = ir_colormapper()

In [7]:
local_feature_annotator = feature_annotators.get_plugin("default")

local_gridline_annotator = gridline_annotators.get_plugin("default")

In [8]:
# imagery_annotated
img_ann_output_formatter = output_formatters.get_plugin("imagery_annotated")

formatter_result_dict = {}

timestamp = datetime.strftime(datetime.now(), "%Y%m%d%H%M%S")
out_fpath = f"{os.environ['GEOIPS_OUTDIRS']}/abi_infrared_annotated_test_{timestamp}.png"

success_outputs = img_ann_output_formatter(conus_sector_area, output_dataset, "Infrared", [out_fpath], mpl_colors_info=ir_color_dict, feature_annotator=local_feature_annotator, gridline_annotator=local_gridline_annotator, output_dict=formatter_result_dict)
success_outputs

['/home/coleman/geoips/outdirs/abi_infrared_annotated_test_20250610102248.png']

In [9]:
img_clean_output_formatter = output_formatters.get_plugin("imagery_clean")

out_fpath = f"{os.environ['GEOIPS_OUTDIRS']}/abi_infrared_clean_test_{timestamp}.png"

success_outputs = img_clean_output_formatter(conus_sector_area, output_dataset, "Infrared", [out_fpath], mpl_colors_info=ir_color_dict)
success_outputs

['/home/coleman/geoips/outdirs/abi_infrared_clean_test_20250610102248.png']

In [16]:
ncdf_output_formatter = output_formatters.get_plugin("netcdf_geoips")

out_fpath = f"{os.environ['GEOIPS_OUTDIRS']}/abi_infrared_xarray_test_{timestamp}.nc"

success_outputs = ncdf_output_formatter(output_dataset, ["Infrared"], [out_fpath])
success_outputs

SKIPPING not outputing file /home/coleman/geoips/outdirs/abi_infrared_xarray_test_20250610102248.nc, exists


['/home/coleman/geoips/outdirs/abi_infrared_xarray_test_20250610102248.nc']